In [174]:
import os
import RAKE
import nltk
import numpy as np
from summa import keywords
from string import punctuation
from pymystem3 import Mystem
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer
from pymorphy2.tokenizers import simple_word_tokenize

# 1. Подобрать корпус

В качестве мини-корпуса были выбраны несколько недавних статей с сайта Школы Лингвистики. У них есть теги и список организаций, которые упомянуты в статье.

# 2. Разметить КС

Критерии:

- тэги, которых нет в тексте, удаляются
- если новость о мероприятии, добавляется название типа мероприятия (конференция, хакатон) и "онлайн", если мероприятие происходит онлайн
- если есть определенная научная сфера, которая определяет мероприятие, и эта сфера упоминается в тексте, название добавляется в список КС
- места проведения мероприятия и другие задействованные огранизации тоже добавляются в список КС

Теперь сделаем так, чтобы было удобно работать с корпусом. Пишем функцию препроцессинга:

In [95]:
m = MorphAnalyzer()
def preprocess(text):
    lemmas = []
    for t in simple_word_tokenize(text):
        lemmas.append(
            m.parse(t)[0].normal_form
        )
    return ' '.join(lemmas)

Загружаем корпус:

In [120]:
texts = []
kws = []

for file in os.listdir("corpus"):
    if os.path.splitext(os.path.join("corpus", file))[1] == ".txt":
        with open(os.path.join("corpus", file), encoding="utf-8") as f:
            lines = f.readlines()
            texts.append("\n".join(lines))
            kws.append(lines[-1].split(", "))

Сразу препроцессим ключевые слова:

In [214]:
kws_preproc = [[preprocess(word) for word in kw_list] for kw_list in kws]

In [215]:
kws_preproc

[['экспедиция', 'шетнево', 'макеево', 'деревня', 'селигеро-торжковский говор'],
 ['поступление',
  'магистратура',
  'бакалавриат',
  'программа',
  'образовательный программа',
  'ниу вшэ'],
 ['хакатон', 'цифровой гуманитарный исследование', 'dh', 'онлайн'],
 ['конференция', 'компьютерный лингвистика', 'школа лингвистика']]

In [122]:
stop = stopwords.words('russian')

# 3. Применить методы

## RAKE

In [123]:
rake = RAKE.Rake(stop)

In [170]:
rake_kw_lists = []

In [171]:
for text in texts:
    rake_kw_list = rake.run(preprocess(text), maxWords=3, minFrequency=2)
    rake_kw_lists.append([elem[0] for elem in rake_kw_list])

## Summa

In [172]:
summa_kw_lists = []

In [173]:
for text in texts:
    summa_kw_list = keywords.keywords(normalize_text(text),
                                      language='russian',
                                      additional_stopwords=stop,
                                      scores=True)
    summa_kw_lists.append([elem[0] for elem in summa_kw_list])

## TF-IDF

In [216]:
tfidf = TfidfVectorizer(ngram_range=(1,3))

In [217]:
tfidf_matrix = tfidf.fit_transform([preprocess(text) for text in texts])

In [218]:
tfidf_lists = []
for row in tfidf_matrix:
    tfidf_kws_ix = np.argsort(row.toarray())[0][:-11:-1]
    tfidf_lists.append(np.array(tfidf.get_feature_names())[tfidf_kws_ix])

/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


# 4. Шаблоны

## Создаем

In [129]:
kws_flattened = [item for sublist in kws for item in sublist]

In [223]:
def to_tag(text):
    postag = ""
    for t in simple_word_tokenize(text):
        if m.parse(t)[0].tag.POS:
            postag += m.parse(t)[0].tag.POS+"+"
    return postag[:-1]

In [224]:
postags = []
for elem in kws_flattened:
    postags.append(to_tag(elem))

In [225]:
postags

['NOUN',
 'NOUN',
 'NOUN',
 'NOUN',
 'NOUN+NOUN',
 'NOUN',
 'NOUN',
 'NOUN',
 'NOUN',
 'ADJF+NOUN',
 '',
 'NOUN',
 'ADJF+ADJF+NOUN',
 '',
 'ADVB',
 'NOUN',
 'ADJF+NOUN',
 'NOUN+NOUN']

In [226]:
postags_unique = set(postags)
postags_unique.remove("")

In [227]:
postags_unique

{'ADJF+ADJF+NOUN', 'ADJF+NOUN', 'ADVB', 'NOUN', 'NOUN+NOUN'}

## Фильтруем

In [222]:
kw_suggestions = [rake_kw_lists,
                  summa_kw_lists,
                  tfidf_lists]

In [238]:
filtered_suggestions = []
for kw_lists in kw_suggestions:
    filtered_lists = []
    for kw_list in kw_lists:
        filtered_list = list(filter(lambda x: to_tag(x) in postags_unique,
                                    kw_list))
        filtered_lists.append(filtered_list)
    filtered_suggestions.append(filtered_lists)

In [239]:
filtered_suggestions

[[['селигеро-торжковский говор',
   'смоленский говор',
   'иной житель',
   'деревня шетнево',
   'деревня',
   'дом',
   'кличка',
   'шетнево',
   'озеро',
   'обед',
   'экспедиция',
   'информант',
   'почему',
   'макеево',
   'поэтому',
   'семья',
   'прозвище',
   'поколение',
   'целое'],
  ['вступительный испытание',
   'результат егэ',
   'подача документ',
   'бюджетный место',
   'санкт-петербург',
   'нижний новгород',
   'русский язык',
   'прошлое год',
   'английский язык',
   'программа',
   'вышка',
   'онлайн',
   'конкурс',
   '2022 год',
   '2021 год',
   '« магистр',
   'бакалавриат',
   'зачисление',
   'поступление',
   'магистратура',
   '20 июнь',
   'е',
   'математика',
   'число',
   'москва',
   '« кибербезопасность »',
   'наука'],
  ['цифровой гуманитарный исследование'],
  ['конференция', 'эссе']],
 [['кличка',
   'диалектологический экспедиция',
   'мочь',
   'говор',
   'время',
   'местный житель',
   'макеево',
   'особенно',
   'особенность',
   

# 5. Оценить точность

In [261]:
def fscore(prec, rec):
    return 2 * prec * rec / (prec + rec)

scores = []
for sugg in filtered_suggestions:
    algo_score = []
    for i, kws in enumerate(sugg):
        overall_kws = list(filter(lambda x: x in kws_preproc[i], kws))
        precision = len(overall_kws)/len(kws)
        recall = len(overall_kws)/len(kws_preproc[i])
        algo_score.append((precision,
                           recall,
                           fscore(precision, recall)))
    scores.append(algo_score)

In [279]:
mean_scores = np.array(scores).mean(axis=1)

In [283]:
methods = ("RAKE", "SUMMA", "TFIDF")
print("method\tprec\trec\tf-score")
for i, row in enumerate(mean_scores):
    print(f"{methods[i]}\t{row[0]:.3f}\t{row[1]:.3f}\t{row[2]:.3f}")

method	prec	rec	f-score
RAKE	0.478	0.562	0.365
SUMMA	0.092	0.467	0.151
TFIDF	0.300	0.463	0.354


По F-score лучше всего оказывается RAKE, дальше -- TfIdf и потом с огромным отрывом -- TextRank.

# Почему так?

Скорее всего, алгоритм, основанный на встречаемости с другими словами в данном случае работает слабее, чем алгоритм, основанный на простой частотности слова. Сами тексты не достаточно разнообразны для того, чтобы можно было считать слово более "ключевым" в связи с тем, что с ним встречается больше других слов и фраз. Многие изначально обозначенные ключевые слова встречаются всего два раза.

Но также можно винить и изначальную разметку, основанную на интуиции, а не на математических подсчетах.

In [284]:
# made by nejenek